In [34]:
# Scarpe data from https://s3.amazonaws.com/tripdata/JC-201509-citibike-tripdata.csv.zip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os
import glob
import zipfile
import urllib.request
from urllib.error import HTTPError
import shutil
from tqdm import tqdm


In [25]:
# Download data from citibike website
def download_data_one_month(year, month):
    month = str(month).zfill(2)
    url = f'https://s3.amazonaws.com/tripdata/JC-{year}{month}-citibike-tripdata.csv.zip'
    output_path = 'data'
    file_name = f'JC-{year}{month}-citibike-tripdata.csv.zip'
    with urllib.request.urlopen(url) as response, open(os.path.join(output_path, file_name), 'wb') as out_file:
        shutil.copyfileobj(response, out_file)
    with zipfile.ZipFile(os.path.join('data', file_name), 'r') as zip_ref:
        zip_ref.extractall('data')
    os.remove(os.path.join('data', file_name))
    return f'JC-{year}{month}-citibike-tripdata.csv'

download_data_one_month(2015, 10)

'JC-201510-citibike-tripdata.csv'

In [28]:
def download_all_data():
    dfs = []
    # Limiting to 2016-2024 to get full years of data
    for year in tqdm(range(2016, 2024), desc='Year'):
        for month in range(1, 13):
            try:
                download_data_one_month(year, month)
            except HTTPError as e:
                print(f'Failed to download file for {year}-{month}, {e}')

In [29]:
download_all_data()

Failed to download file for 2015-1, HTTP Error 404: Not Found
Failed to download file for 2015-2, HTTP Error 404: Not Found
Failed to download file for 2015-3, HTTP Error 404: Not Found
Failed to download file for 2015-4, HTTP Error 404: Not Found
Failed to download file for 2015-5, HTTP Error 404: Not Found
Failed to download file for 2015-6, HTTP Error 404: Not Found
Failed to download file for 2015-7, HTTP Error 404: Not Found
Failed to download file for 2015-8, HTTP Error 404: Not Found
Downloaded JC-201509-citibike-tripdata.csv
Downloaded JC-201510-citibike-tripdata.csv
Downloaded JC-201511-citibike-tripdata.csv
Downloaded JC-201512-citibike-tripdata.csv
Downloaded JC-201601-citibike-tripdata.csv
Downloaded JC-201602-citibike-tripdata.csv
Downloaded JC-201603-citibike-tripdata.csv
Downloaded JC-201604-citibike-tripdata.csv
Downloaded JC-201605-citibike-tripdata.csv
Downloaded JC-201606-citibike-tripdata.csv
Downloaded JC-201607-citibike-tripdata.csv
Downloaded JC-201608-citibike-t

For the months that raised an error, I manually downloaded the files and put them in the data folder. Those months are:
- 2017-08 : missing a dash in the file name
- 2022-07 : typo in "citibike"

In [39]:
def standardize_columns(df):
    df.columns = df.columns.str.lower()
    col_to_rename = {'starttime': 'start time', 'stoptime': 'stop time', 'tripduration': 'trip duration', 'bikeid':'bike id', 'usertype': 'user type',
                     'started_at': 'start time', 'ended_at': 'stop time',
                     'start_lat': 'start station latitude', 'start_lng': 'start station longitude', 
                     'end_lat': 'end station latitude', 'end_lng': 'end station longitude'}
    df = df.rename(columns=col_to_rename)
    cols = ['_'.join(col.split()) for col in df.columns]
    df.columns = cols
    return df


def load_data():
    for file in tqdm(glob.glob(os.path.join('data', '*.csv'))):
        df = pd.read_csv(file).pipe(standardize_columns)
        yield df
        
# Loading 12 times 8 years of data
df = pd.concat(load_data())
df

100%|██████████| 99/99 [00:18<00:00,  5.29it/s]


,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,birth_year,gender,ride_id,rideable_type,member_casual
0,148.0,2017-01-01 00:21:32,2017-01-01 00:24:01,3276,Marin Light Rail,40.714584,-74.042817,3185,City Hall,40.717732,-74.043845,24575.0,Subscriber,1983.0,1.0,NaN,NaN,NaN
1,1283.0,2017-01-01 00:24:35,2017-01-01 00:45:58,3183,Exchange Place,40.716247,-74.033459,3198,Heights Elevator,40.748716,-74.040443,24723.0,Subscriber,1978.0,1.0,NaN,NaN,NaN
2,372.0,2017-01-01 00:38:19,2017-01-01 00:44:31,3183,Exchange Place,40.716247,-74.033459,3211,Newark Ave,40.721525,-74.046305,24620.0,Subscriber,1989.0,1.0,NaN,NaN,NaN
3,1513.0,2017-01-01 00:38:37,2017-01-01 01:03:50,3194,McGinley Square,40.725340,-74.067622,3271,Danforth Light Rail,40.692640,-74.088012,24668.0,Subscriber,1961.0,1.0,NaN,NaN,NaN
4,639.0,2017-01-01 01:47:52,2017-01-01 01:58:31,3183,Exchange Place,40.716247,-74.033459,3203,Hamilton Park,40.727596,-74.044247,26167.0,Subscriber,1993.0,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53828,270.0,2020-10-13 15:05:35.0860,2020-10-13 15:10:05.3430,3207,Oakland Ave,40.737604,-74.052478,3640,Journal Square,40.733670,-74.062500,44744.0,Subscriber,1963.0,2.0,NaN,NaN,NaN
53829,400.0,2020-10-13 15:09:03.4890,2020-10-13 15:15:43.9750,3209,Brunswick St,40.724176,-74.050656,3209,Brunswick St,40.724176,-74.050656,45345.0,Subscriber,1984.0,1.0,NaN,NaN,NaN
53830,206.0,2020-10-13 15:11:34.3500,2020-10-13 15:15:00.5030,3195,Sip Ave,40.730897,-74.063913,3194,McGinley Square,40.725340,-74.067622,47019.0,Subscriber,1993.0,1.0,NaN,NaN,NaN
53831,216.0,2020-10-13 15:11:49.1510,2020-10-13 15:15:25.6930,3195,Sip Ave,40.730897,-74.063913,3225,Baldwin at Montgomery,40.723659,-74.064194,42191.0,Subscriber,1966.0,1.0,NaN,NaN,NaN


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4207977 entries, 0 to 53832
Data columns (total 18 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   trip_duration            float64
 1   start_time               object 
 2   stop_time                object 
 3   start_station_id         object 
 4   start_station_name       object 
 5   start_station_latitude   float64
 6   start_station_longitude  float64
 7   end_station_id           object 
 8   end_station_name         object 
 9   end_station_latitude     float64
 10  end_station_longitude    float64
 11  bike_id                  float64
 12  user_type                object 
 13  birth_year               float64
 14  gender                   float64
 15  ride_id                  object 
 16  rideable_type            object 
 17  member_casual            object 
dtypes: float64(8), object(10)
memory usage: 610.0+ MB


In [42]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df, title='Citi Bike Data Profiling Report', explorative=True)
profile.to_file('data_report/citi_bike_data_profiling_report.html')


In [47]:
df.to_csv('data/citi_bike_data.csv', index=False)
